In [22]:
# Active Learning from scratch run
# based on random sampling by increasing dataset step by step
# Loop over all al_steps with local evaluation

In [12]:
# Cloud install
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install seqeval

In [1]:
# import data
from datasets import load_dataset

# raw_datasets = load_dataset("conll2003") # attention: conll2003 is already tokenized into words
# raw_datasets = load_dataset("limsc/requirements-entity-recognition")
raw_datasets = load_dataset("dxiao/requirements-ner")


Using custom data configuration dxiao--requirements-ner-0cad378f7506b371
Found cached dataset json (C:/Users/dekai/.cache/huggingface/datasets/dxiao___json/dxiao--requirements-ner-0cad378f7506b371/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
# get label names
ner_feature = raw_datasets["train"].features["ner_tags"]
label_names = ner_feature.feature.names
label_names

['O',
 'B-ACT',
 'I-ACT',
 'B-ATTR',
 'I-ATTR',
 'B-RELOP',
 'I-RELOP',
 'B-QUANT',
 'I-QUANT',
 'B-ENT',
 'I-ENT']

In [3]:
# import transformer
from transformers import AutoTokenizer

model_config = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_config)

## Preprocessing

In [4]:
# tokenize pre-tokenized input (due to conll2003 being already split into words)
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()
# because "lamb" has been split, realignment is needed 

['[CLS]',
 'The',
 'operating',
 'humidity',
 'shall',
 'be',
 'between',
 '0',
 '.',
 '4',
 'and',
 '0',
 '.',
 '6',
 '[SEP]']

In [5]:
# example before alignment
print("labels:   " + str(raw_datasets["train"][0]["ner_tags"]))
print("word_ids: " + str(inputs.word_ids()))

labels:   [0, 3, 4, 0, 1, 5, 7, 0, 7]
word_ids: [None, 0, 1, 2, 3, 4, 5, 6, 6, 6, 7, 8, 8, 8, None]


In [6]:
# alignment function
# special tokens = -100
# replace tokens with have been split from B into B + I

def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word_id = None
    for word_id in word_ids:
        if word_id != current_word_id: # check if it is not same word, just split into two
            current_word_id = word_id 
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [7]:
# test of alignment
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 3, 4, 0, 1, 5, 7, 0, 7]
[-100, 0, 3, 4, 0, 1, 5, 7, 8, 8, 0, 7, 8, 8, -100]


In [8]:
# scaling align_labels_with_tokens function towards entire datasets by using lists in lists [[]]

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True) # similar to word_ids

    all_labels = examples["ner_tags"] # similar to labels but recursive [[ner_tag1],[ner_tag2],[ner_tag3]]
    new_labels = []
    for i, labels in enumerate(all_labels): # i = iterative number, labels = [ner_tag1]
        word_ids = tokenized_inputs.word_ids(i) # takes word_ids from tokenizer magic based on iterative number 
        new_labels.append(align_labels_with_tokens(labels, word_ids)) # creates [[new_labels1],[new_labels2],[new_labels3]]

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [9]:
# batched mapping
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets['train'].column_names,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Dataset slicing

In [10]:
# Slicing tokenized datasets into 20 chunks with each 500 sentences
# For shao: 646 tokens -> 10 steps â 60 

sliced_datasets = {}

for i in range(10): 
    sliced_datasets[i] = tokenized_datasets['train'].select(range(0,(i+1)*60)) # sliced_datasets[0] = first 10%


## Finetuning

In [11]:
# import DataCollator -> adds padding
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [12]:
# test data collator
batch = data_collator([tokenized_datasets['train'][i] for i in range(2)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    0,    3,    4,    0,    1,    5,    7,    8,    8,    0,    7,
            8,    8, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100],
        [-100,    0,    9,   10,   10,   10,   10,   10,   10,   10,   10,   10,
           10,   10,   10,   10,   10,   10,    0,    1,    0,    3,    4,    4,
            4,    4,    0,    5,    7,    8,    8,    8,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0, -100]])

In [13]:
import evaluate

metric = evaluate.load("seqeval")

In [14]:
# seqeval requires inputs in form of String lists -> give labels as string
labels = raw_datasets['train'][10]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'ACT': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ATTR': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ENT': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'QUANT': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'RELOP': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 0.6666666666666666,
 'overall_recall': 0.8,
 'overall_f1': 0.7272727272727272,
 'overall_accuracy': 0.967741935483871}

In [15]:
# write a function to strip special tokens and convert to strings -> true_labels + true_predictions
# compare true_labels with true_predictions using seqeval

import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] # single labels
        for label in labels] # scale towards every label in sentence
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100] # single labels, zip with l in order to delete -100 tokens
        for prediction, label in zip(predictions, labels) # scale towards every label in sentence
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

## Define the model

In [16]:
# dictionarys for better mapping
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [18]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    'limsc/reqroberta-tapt-epoch43',
    from_tf = True,
    id2label=id2label,
    label2id=label2id,
)

All TF 2.0 model weights were used when initializing RobertaForTokenClassification.

All the weights of RobertaForTokenClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaForTokenClassification for predictions without further training.


In [19]:
# Train and save models over all al_steps locally

from transformers import TrainingArguments
from transformers import Trainer

model_path_list = []

for al_step in range(1): # loop over all AL iteration steps [0 = 10%, 1=20%, etc.]

    args = TrainingArguments(
        "cache/bert-finetuned-ner-" + str((al_step+1)*10) + "percent", # al_step = 0 -> 10% data
        evaluation_strategy="epoch",
        save_strategy="no",
        learning_rate=2e-5,
        num_train_epochs=3,
        weight_decay=0.01,
        push_to_hub=False,
        seed = 2022
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=sliced_datasets[al_step],
        eval_dataset=tokenized_datasets["val"],
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
    )
    trainer.train()

    model_path = "./bert-finetuned-ner-" + str((al_step+1)*10) + "percent"
    model_path_list.append(model_path)
    trainer.save_model(model_path)

c:\Users\dekai\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 60
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 24


  0%|          | 0/24 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Evaluation of local model

In [1]:
import pandas as pd
from datasets import load_dataset
from evaluate import evaluator
from transformers import pipeline

In [ ]:
# evaluate model directly from local path

# dataset_test = load_dataset("conll2003", split="test").select(range(1000)).shuffle()
dataset_test = load_dataset("limsc/requirements-entity-recognition", split="test")
task_evaluator = evaluator("token-classification")

results = []
for model in model_path_list:
    results.append(
        task_evaluator.compute(
            model_or_pipeline=model, data=dataset_test, metric="seqeval"
            )
        )

df = pd.DataFrame(results, index=model_path_list)
df[["overall_f1", "overall_accuracy", "total_time_in_seconds", "samples_per_second", "latency_in_seconds"]]

In [ ]:
# trainer.push_to_hub(commit_message="Training complete")